<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN08/8.1-Was_CNN_lernen_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Visualisierung: Was CNNs lernen (towards understandable AI)

This notebook contains the code sample found in Chapter 5, Section 4 of [Deep Learning with Python] (https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff) and
[Deep Learning mit Python](http://www.mitp.de/Alle-Buecher-7/Deep-Learning-mit-Python-und-Keras.html)



Oft heisst es, Deep-Learning-Modelle seien »Blackboxes«, die Repräsentationen in einer Form erlernen, die schwer zu extrahieren und für Menschen kaum verständlich sind. 
- Für einige Arten von Deep-Learning-Modellen mag das teilweise stimmen, auf CNNs trifft es jedoch definitiv nicht zu. 
- Die von CNNs erlernten Repräsentationen sind in hohem Mass für Visualisierungen geeignet, und zwar vor allem deshalb, weil es sich um Repräsentationen visueller Konzepte handelt. Seit 2013 wurde eine Vielzahl von Verfahren zur Visualisierung und Interpretation dieser Repräsentationen entwickelt. Wir werden nicht alle diese Verfahren betrachten, aber die drei verständlichsten und nützlichsten erörtern:

Abgesehen von der Befürchtung, dass böse künstliche Intelligenzen die Welt übernehmen werden, kann das Gebiet der künstlichen Intelligenz für Außenstehende entmutigend sein. Facebooks Direktor für künstliche Intelligenz, **Yann LeCun**, verwendet die Analogie, dass KI eine Blackbox mit einer Million Knöpfen ist; das Innenleben ist den meisten ein Rätsel. Aber jetzt haben wir einen Blick hinein geworfen.

**Adam Harley**, Masterstudent an der Ryerson University, hat eine interaktive Visualisierung erstellt, die erklärt, wie ein neuronales Faltungsnetz, eine Art Programm für künstliche Intelligenz, das zur Analyse von Bildern verwendet wird, intern funktioniert.

[Visualization of a CNN](http://www.cs.cmu.edu/~aharley/nn_vis/cnn/3d.html)


In diesem Notebook zeigen wir drei Ansätze, um Einblicke in das Innenleben von Convolutional Neural Networks (CNNs) zu erhalten:

1. **Visualisierung zwischenliegender Aktivierungen** – Wir betrachten, wie ein kleines CNN (hier zur Klassifikation von Katzen und Hunden) verschiedene Eingabemerkmale extrahiert.
2. **Visualisierung der CNN-Filter** – Mittels Gradientenanstieg im Eingaberaum ermitteln wir, welche visuellen Muster einen bestimmten Filter maximal aktivieren.
3. **Grad‑CAM** – Wir berechnen Heatmaps, die zeigen, welche Bereiche eines Bildes massgeblich zur Klassifikationsentscheidung beitragen.
## 0. Setup und Versionsabfrage

Hier laden wir zunächst die nötigen Bibliotheken, prüfen die Lightning-Version und importieren alle Module, die wir später brauchen.

In [ ]:
import lightning as L

print("Lightning Version:", L.__version__)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
import matplotlib.cm as cm


## 1. Visualisierung zwischenliegender Aktivierungen
In diesem Abschnitt wird gezeigt, wie die Aktivierungen (Outputs) der einzelnen Schichten eines CNNs visualisiert werden können, um zu verstehen, welche Merkmale das Modell in den verschiedenen Schichten extrahiert.
### 1.1 Definition eines einfachen CNN als LightningModule

Wir definieren ein kleines CNN, das Katzen von Hunden unterscheidet. Neben der Vorhersage gibt uns die `forward`-Methode auch Zugriff auf die Aktivierungen der einzelnen Layer.


In [ ]:
class CNNModelWithActivations(L.LightningModule):
    def __init__(self):
        super(CNNModelWithActivations, self).__init__()

        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0)
        self.conv4 = torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0)

        self.flatten = torch.nn.Flatten()
        self.dropout = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(128 * 7 * 7, 512)  # Adjusted for input size
        self.fc2 = torch.nn.Linear(512, 1)
        self.sigmoid = torch.nn.Sigmoid()

        self.criterion = torch.nn.BCEWithLogitsLoss()

    def forward(self, x, return_activations=False):
        activations = {}

        x = torch.relu(self.conv1(x))
        if return_activations:
            activations["conv1"] = x
        x = self.pool(x)

        x = torch.relu(self.conv2(x))
        if return_activations:
            activations["conv2"] = x
        x = self.pool(x)

        x = torch.relu(self.conv3(x))
        if return_activations:
            activations["conv3"] = x
        x = self.pool(x)

        x = torch.relu(self.conv4(x))
        if return_activations:
            activations["conv4"] = x
        x = self.pool(x)

        x = self.flatten(x)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        if return_activations:
            return x, activations
        return x


# Modell laden mit Gewichten laden welches in Lektion 6 Trainiert wurde für Hunde und Katzen
model = CNNModelWithActivations()
model.load_state_dict(torch.load("cats_and_dogs_small_2.pth"))  # Pfad zu den Weights


### Instanzierung und Wechsel in den Evaluationsmodus

In [ ]:
model = CNNModelWithActivations()
model.eval()


### 1.2 Bildvorverarbeitung und Anzeige

Wir laden ein Katzenbild und bereiten es so vor, dass es als Eingabe in unser Modell passt. Die Transformationen umfassen das Ändern der Größe auf 150x150 und die Umwandlung in einen Tensor (Skalierung der Pixelwerte auf [0,1]).


In [ ]:
# Bitte passe den Pfad zu deinem Katzenbild an.
cat_img_path = "Bilder/cat.jpg"
img = Image.open(cat_img_path).convert("RGB")

# Transformation: Grösse 150x150, Umwandlung in Tensor
transform = transforms.Compose([transforms.Resize((150, 150)), transforms.ToTensor()])
img_tensor = transform(img).unsqueeze(0)  # Shape: (1, 3, 150, 150)

# Anzeige des Originalbildes
plt.figure()
plt.imshow(img)
plt.title("Originalbild (Katze)")
plt.axis("off")
plt.show()


### 1.3 Abruf und Anzeige der Zwischenaktivierungen

Wir lassen das Bild durch unser CNN laufen und speichern die Ausgaben der verschiedenen Layer. Anschliessend zeigen wir beispielhaft einzelne Kanäle (zum Beispiel Kanal 6 und 7) aus dem ersten Convolutional Layer.


In [ ]:
with torch.no_grad():
    logits, activations = model(img_tensor, return_activations=True)
print("Logits:", logits)


In [ ]:
# Anzeige einzelner Kanäle des ersten Convolutional Layers (conv1)
act_conv1 = activations["conv1"]  # Shape: (1, 32, 150, 150)

plt.figure()
plt.matshow(act_conv1[0, 6].cpu(), cmap="viridis")
plt.title("Aktivierung, conv1 Kanal 6")
plt.axis("off")
plt.show()

plt.figure()
plt.matshow(act_conv1[0, 7].cpu(), cmap="viridis")
plt.title("Aktivierung, conv1 Kanal 7")
plt.axis("off")
plt.show()


In [ ]:
# Anzeige aller Kanäle des ersten Convolutional Layers
for k in range(0, act_conv1.shape[1], 10):
    plt.figure()
    plt.matshow(act_conv1[0, k].cpu(), cmap="viridis")
    plt.title(f"conv1, Kanal {k}")
    plt.axis("off")
    plt.show()


### 1.4 Visualisierung als Raster

Mit der Funktion `display_activation_grid` erstellen wir ein Raster, in dem alle Kanäle eines bestimmten Layers (2D-Aktivierungskarten) angeordnet werden.


In [ ]:
def display_activation_grid(activation, layer_name, images_per_row=8):
    # Erwartete Form: (1, num_channels, H, W)
    activation = activation.cpu().numpy()[0]  # (num_channels, H, W)
    n_features = activation.shape[0]
    size = activation.shape[1]
    n_cols = n_features // images_per_row
    display_grid = np.zeros((n_cols * size, images_per_row * size))
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_index = col * images_per_row + row
            if channel_index >= n_features:
                break
            channel_image = activation[channel_index]
            # Postprocessing: Normalisierung der Aktivierungskarte
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std() + 1e-5
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype("uint8")
            display_grid[
                col * size : (col + 1) * size, row * size : (row + 1) * size
            ] = channel_image
    scale = 1.0 / size
    plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect="auto", cmap="viridis")
    plt.axis("off")
    plt.show()


In [ ]:
# Ausgabe der Rasters für alle Layer des Modells
for name, act in activations.items():
    display_activation_grid(act, layer_name=name, images_per_row=16)


## 2. Visualisierung von CNN-Filtern per Gradientenanstieg

Im folgenden Abschnitt nutzen wir einen vortrainierten VGG16 aus Torchvision. Ziel ist es, ein Bild zu generieren, das einen bestimmten Filter in einem gewählten Layer maximal aktiviert.

Dazu initialisieren wir ein zufälliges Bild (weißes Rauschen) und passen es mittels Gradientenanstieg an, sodass der Mittelwert der Aktivierung des Ziel-Filters maximiert wird.


In [ ]:
def generate_filter_pattern(
    model,
    layer,
    filter_index,
    iterations=50,
    learning_rate=10,
    img_size=200,
):
    """
    Erzeugt ein Bild, das einen bestimmten Filter in `layer` maximal aktiviert.
    """
    # Überprüfen, ob eine GPU verfügbar ist, und das entsprechende Gerät auswählen
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # Initialisiere ein zufälliges Bild (Werte in [0,1])
    input_img = torch.rand(1, 3, img_size, img_size, device=device, requires_grad=True)
    optimizer = torch.optim.Adam([input_img], lr=learning_rate)

    for i in range(iterations):
        optimizer.zero_grad()

        # Verwenden eines Forward-Hooks, um die Aktivierung des gewünschten Layers zu erhalten
        activation = None

        def hook_fn(module, inp, outp):
            nonlocal activation
            activation = outp

        hook = layer.register_forward_hook(hook_fn)
        model(input_img)
        hook.remove()

        # Verlustfunktion: Negative des Mittelwerts der Zielaktivierung (wir minimieren, um zu maximieren)
        loss = -activation[0, filter_index].mean()
        loss.backward()
        optimizer.step()
        # Optional: Fortschritt ausgeben
        # print(f"Iteration {i}: Loss = {-loss.item():.4f}")

    result = input_img.detach().squeeze().cpu()
    # Postprocessing: Normalisierung und Transformation in einen uint8-Bildbereich
    result = result - result.mean()
    result = result / (result.std() + 1e-5)
    result = result * 64 + 128
    result = torch.clamp(result, 0, 255)
    result = result.permute(1, 2, 0).numpy().astype("uint8")
    return result


In [ ]:
from torchvision.models import VGG16_Weights

# Laden eines vortrainierten VGG16
vgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT).to("cpu")
vgg16.eval()
# Wähle einen Conv-Layer aus dem Feature-Extractor. Hier verwenden wir beispielhaft features[10].
target_layer = vgg16.features[10]


In [ ]:
# Visualisierung des Filtermusters für einen Beispiel-Filter (hier: Filter 2)
pattern = generate_filter_pattern(
    vgg16,
    target_layer,
    filter_index=2,
    iterations=50,
    learning_rate=10,
    img_size=200,
)
plt.figure()
plt.imshow(pattern)
plt.title("Filtervisualisierung: Filter 2")
plt.axis("off")
plt.show()


### 2.1 Erzeugen eines Rasters mit den ersten 64 Filtern

Wir generieren die Filtermuster für die ersten 64 Filter und fügen diese in einem Raster zusammen, wobei zwischen den Bildern ein kleiner Rand (margin) eingehalten wird.


In [ ]:
all_images = []
for filter_index in range(64):
    print(f"Verarbeite Filter {filter_index}")
    img_pattern = generate_filter_pattern(
        vgg16,
        target_layer,
        filter_index=filter_index,
        iterations=50,
        learning_rate=10,
        img_size=200,
    )
    all_images.append(img_pattern)

margin = 5
n = 8
img_h, img_w, _ = all_images[0].shape
grid_h = n * img_h + (n - 1) * margin
grid_w = n * img_w + (n - 1) * margin
stitched_filters = np.zeros((grid_h, grid_w, 3), dtype=np.uint8)

for i in range(n):
    for j in range(n):
        stitched_filters[
            i * (img_h + margin) : i * (img_h + margin) + img_h,
            j * (img_w + margin) : j * (img_w + margin) + img_w,
        ] = all_images[i * n + j]

plt.figure(figsize=(16, 16))
plt.imshow(stitched_filters)
plt.title("Raster der ersten 64 Filter")
plt.axis("off")
plt.show()


In [ ]:
# Speichern des Filter-Rasters als Bilddatei
from PIL import Image as PILImage
import requests

pil_img = PILImage.fromarray(stitched_filters)
pil_img.save("filters_for_layer.png", dpi=(600, 600))
print("Filter-Raster gespeichert als 'filters_for_layer.png'.")


## 3. Visualisierung der Heatmaps der Klassenaktivierung (Grad‑CAM)

Im letzten Abschnitt demonstrieren wir Grad‑CAM. Dabei wird mithilfe von Hook-Funktionen der Einfluss einzelner Pixelbereiche
auf die Klassifikation sichtbar gemacht. Wir berechnen die Gradienten im Ziel-Layer und gewichten die Aktivierungen, um eine Heatmap zu erhalten, die anschließend auf das Originalbild gelegt wird.


In [ ]:
def grad_cam(model, img_tensor, target_class, target_layer):
    """
    Berechnet die Grad‑CAM Heatmap für das gegebene Bild.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    img_tensor = img_tensor.to(device)

    model.eval()
    activation = None
    gradient = None

    def forward_hook(module, inp, outp):
        nonlocal activation
        activation = outp.detach()

    def backward_hook(module, grad_in, grad_out):
        nonlocal gradient
        gradient = grad_out[0].detach()

    hook_forward = target_layer.register_forward_hook(forward_hook)
    hook_backward = target_layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    # Falls target_class als Index übergeben wird:
    score = (
        output[0, target_class]
        if isinstance(target_class, int)
        else output[0, output.argmax()]
    )
    model.zero_grad()
    score.backward()

    hook_forward.remove()
    hook_backward.remove()

    # Globales Durchschnittspooling der Gradienten über räumliche Dimensionen
    weights = torch.mean(gradient, dim=(2, 3))  # (batch, channels)
    cam = torch.zeros(activation.shape[2:], dtype=torch.float32, device=device)
    for i, w in enumerate(weights[0]):
        cam += w * activation[0, i, :, :]
    cam = torch.relu(cam)
    cam = cam - cam.min()
    if cam.max() != 0:
        cam = cam / cam.max()
    return cam.cpu().numpy()


### 3.1 Bildvorbereitung für Grad‑CAM

Wir laden ein Elefantenbild, transformieren es auf 224×224 Pixel und normalisieren es gemäss den ImageNet-Standards. So passt es zum vortrainierten VGG16-Modell.


In [ ]:
# Bitte passe den Pfad zu deinem Elefantenbild an.
elephant_img_path = "Bilder/elefant.jpg"
img_elephant = Image.open(elephant_img_path).convert("RGB")
preprocess = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
img_tensor_vgg = preprocess(img_elephant).unsqueeze(0)


In [ ]:
# Vorhersage mit VGG16 und Bestimmung der Zielklasse
device = next(vgg16.parameters()).device  # Get the device of the model
img_tensor_vgg = img_tensor_vgg.to(
    device
)  # Move the input tensor to the same device as the model
output = vgg16(img_tensor_vgg)
preds = torch.nn.functional.softmax(output, dim=1)
pred_class = preds.argmax(dim=1).item()

# Herunterladen der ImageNet-Klassen
imagenet_classes_url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
imagenet_classes = requests.get(imagenet_classes_url).json()

# Ausgabe der Top-3-Klassen
top3_probs, top3_indices = torch.topk(preds, 3)
top3_probs = top3_probs[0].tolist()
top3_indices = top3_indices[0].tolist()

print("Top-3 Vorhersagen:")
for i, (prob, idx) in enumerate(zip(top3_probs, top3_indices)):
    print(f"{i + 1}. {imagenet_classes[idx]} ({prob * 100:.2f}%)")


### 3.2 Grad‑CAM Berechnung und Visualisierung

Wir wählen als Ziel-Layer für Grad‑CAM den letzten Convolutional Layer im Feature-Extractor.
Bei VGG16 eignet sich hierfür beispielsweise `features[28]`. Die berechnete Heatmap wird anschliessend auf das Originalbild gelegt.


In [ ]:
target_layer_cam = vgg16.features[28]
cam = grad_cam(
    vgg16, img_tensor_vgg, target_class=pred_class, target_layer=target_layer_cam
)


In [ ]:
import matplotlib

# Erzeugen der Heatmap und Überlagerung auf das Originalbild
# Wir passen die Grösse der Heatmap an das Originalbild an, wandeln sie in einen uint8-Bereich um und verwenden dann die "jet"-Colormap.
cam_resized = cv2.resize(cam, (img_elephant.size[0], img_elephant.size[1]))
cam_resized = np.uint8(255 * cam_resized)

jet = matplotlib.colormaps.get_cmap("jet")
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[cam_resized]
jet_heatmap = np.uint8(jet_heatmap * 255)

# Umwandlung des Originalbildes in ein Numpy-Array (RGB)
img_np = np.array(img_elephant)
# Überlagerung: 60% Originalbild, 40% Heatmap
superimposed_img = cv2.addWeighted(img_np, 0.6, jet_heatmap, 0.4, 0)

plt.figure(figsize=(16, 9))
plt.imshow(superimposed_img)
plt.title("Grad-CAM Heatmap auf Elefantenbild")
plt.axis("off")
plt.show()


In [ ]:
# Optional: Speichern des Grad‑CAM Ergebnisses
result_img = PILImage.fromarray(superimposed_img)
result_img.save("elephant_gradcam.png", dpi=(600, 600))
print("Grad-CAM Bild gespeichert als 'elephant_gradcam.png'.")
